In [63]:
import os
import io
import uuid
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import base64
from groq import Groq
from dotenv import load_dotenv

In [64]:
DATA_PATH = "AGC_Data.csv"
WINDOW_SIZE = 128
WINDOW_STEP = 64
IMAGE_SIZE = (224, 224)

In [65]:
load_dotenv()
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

In [66]:
def load_data(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path, parse_dates=["indo_time"])
    df = df.sort_values("indo_time").reset_index(drop=True)
    return df

In [67]:
def sliding_windows(values: np.ndarray, window_size: int, step: int):
    n = len(values)
    idx = 0
    while idx + window_size <= n:
        yield idx, idx + window_size, values[idx : idx + window_size]
        idx += step

In [68]:
def normalize_window(window: np.ndarray) -> np.ndarray:
    mu = window.mean()
    sigma = window.std()
    if sigma < 1e-6:
        return np.zeros_like(window)
    return (window - mu) / sigma

In [69]:
def window_to_lineplot_image(window: np.ndarray, size=(224, 224)) -> Image.Image:
    fig, ax = plt.subplots(figsize=(size[0] / 100, size[1] / 100), dpi=100)
    ax.plot(window, linewidth=1)
    ax.set_xticks([]); ax.set_yticks([])
    ax.set_xlim(0, len(window) - 1)
    ax.set_ylim(np.min(window) - 0.1, np.max(window) + 0.1)
    plt.tight_layout(pad=0)
    
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=100, bbox_inches="tight", pad_inches=0)
    plt.close(fig)
    buf.seek(0)
    img = Image.open(buf).convert("RGB")
    img = img.resize(size, Image.BILINEAR)
    return img

In [70]:
def window_to_gaf_image(window: np.ndarray, size=(224, 224)) -> Image.Image:
    minv, maxv = window.min(), window.max()
    if abs(maxv - minv) < 1e-6:
        scaled = np.zeros_like(window)
    else:
        scaled = 2 * (window - minv) / (maxv - minv) - 1
    scaled = np.clip(scaled, -1, 1)
    
    phi = np.arccos(scaled)
    gaf = np.cos(phi[:, None] + phi[None, :])
    
    in_min, in_max = gaf.min(), gaf.max()
    if abs(in_max - in_min) < 1e-6:
        img_arr = np.zeros_like(gaf)
    else:
        img_arr = 255 * (gaf - in_min) / (in_max - in_min)
    img_arr = np.uint8(img_arr)
    
    img = Image.fromarray(img_arr).convert("RGB")
    img = img.resize(size, Image.BILINEAR)
    return img

In [71]:
def upload_image_return_url(img: Image.Image):
    buffered = io.BytesIO()
    if img.mode == 'RGBA':
        img.save(buffered, format="PNG")
    else:
        img.save(buffered, format="JPEG")
    img_byte_arr = buffered.getvalue()
    return base64.b64encode(img_byte_arr).decode('utf-8')

In [72]:
def classify_window_with_groq(base64_image) -> float:
    prompt = [
        {
            "type": "text",
            "text": (
                "You are an industrial anomaly detector. Given the conveyor speed plot image, "
                "output JSON: {\"anomaly\": 1} if you see anomalous slowdown patterns typical of a glass break, "
                "else {\"anomaly\": 0}. "
                "The image is a line plot of conveyor speed over time, "
                "Understand the plot and determine if there is an anomaly. Anomaly don't occur often but when they do, they have a window of points"
                "Answer with exactly one JSON object and no extra text."
            ),
        },
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
        },
    ]

    completion = client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
        max_completion_tokens=64,
        top_p=1.0,
        stream=False,
        response_format={"type": "json_object"},
        stop=None,
    )
    response = completion.choices[0].message
    if isinstance(response, dict) and "anomaly" in response:
        return float(response["anomaly"])
    else:
        return 0.0

In [73]:
def aggregate_anomaly_scores(
    timestamps: np.ndarray,
    window_indices: list,
    scores: list,
    window_size: int,
    window_step: int,
) -> pd.DataFrame:
    N = len(timestamps)
    labels = np.zeros(N, dtype=int)

    for (start_idx, end_idx), score in zip(window_indices, scores):
        if score >= 0.5:
            labels[start_idx : end_idx] = 1

    df = pd.DataFrame({"timestamp": timestamps, "anomaly_flag": labels})
    df["shifted"] = df["anomaly_flag"].shift(1, fill_value=0)
    df["new_segment"] = ((df["anomaly_flag"] == 1) & (df["shifted"] == 0)).astype(int)
    segment_starts = df.loc[df["new_segment"] == 1, "timestamp"].tolist()

    df["next"] = df["anomaly_flag"].shift(-1, fill_value=0)
    segment_ends = df.loc[(df["anomaly_flag"] == 1) & (df["next"] == 0), "timestamp"].tolist()

    intervals = pd.DataFrame({"start_time": segment_starts, "end_time": segment_ends})
    return intervals

In [74]:
def main():
    df = load_data(DATA_PATH).iloc[10000:20000]
    timestamps = df["indo_time"].values
    speeds = df["speed"].values.astype(float)

    window_indices = []
    anomaly_scores = []

    for start_idx, end_idx, raw_window in sliding_windows(speeds, WINDOW_SIZE, WINDOW_STEP):
        norm_win = normalize_window(raw_window)

        img = window_to_lineplot_image(norm_win, size=IMAGE_SIZE)

        try:
            url = upload_image_return_url(img)
        except Exception as e:
            print(f"Error uploading image: {e}") 

        score = classify_window_with_groq(url)
        print(f"Window [{start_idx}:{end_idx}] → anomaly score = {score}")
        window_indices.append((start_idx, end_idx))
        anomaly_scores.append(score)

    intervals = aggregate_anomaly_scores(
        timestamps, window_indices, anomaly_scores, WINDOW_SIZE, WINDOW_STEP
    )

    print("\nDetected anomalous intervals:")
    if intervals.empty:
        print("  (none detected)")
    else:
        for _, row in intervals.iterrows():
            print(f"  • {row.start_time}  →  {row.end_time}")

    intervals.to_csv("detected_anomalies_10000.csv", index=False)
    print("\nSaved intervals to detected_anomalies.csv")

In [75]:
if __name__ == "__main__":
    main()

APITimeoutError: Request timed out.